In [1]:
#!pip install pydotplus

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.tree import export_graphviz
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import graphviz
# import pydotplus
import io
from scipy import misc

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [6]:
data1 = pd.read_csv("/Users/louisabasta/Desktop/Programming 1 assignment/studentAssessmentInfo.csv")
data1 = data1.set_index(['id_student', 'code_module', 'code_presentation','num_of_prev_attempts','final_result'])

# data1.head()
# data1.shape

In [7]:
# data on student assessment
data2 = pd.read_csv("/Users/louisabasta/Desktop/Programming 1 assignment/studentInfoVleQ.csv")
data2 = data2.rename(columns={"code_module_x": "code_module"})
data2 = data2.fillna(0.0)
data2 = data2.set_index(['id_student', 'code_module', 'code_presentation','num_of_prev_attempts','final_result'])
data2.activity_type.value_counts()
# data2.head()
data2.shape

(2021069, 10)

Creating a new dataframe 
# studentAssessmentInfo + studentInfoVleQ

In [8]:
data = pd.merge(data1, data2, right_index=True, left_index=True)
data = data.reset_index()
df = pd.DataFrame(data, columns=['code_module', 'code_presentation', 'final_result',
                                 'assessment_type', 'weight','score', 'gender', 'imd_band', 
                                 'id_site', 'activity_type', 'sum' ])
df.shape


(15261462, 11)

In [9]:
df.describe()
df.imd_band[data.imd_band==np.nan]


Series([], Name: imd_band, dtype: float64)

In [10]:
df.dtypes


code_module            int64
code_presentation      int64
final_result           int64
assessment_type        int64
weight               float64
score                float64
gender                 int64
imd_band             float64
id_site                int64
activity_type          int64
sum                    int64
dtype: object

In [11]:
train, test = train_test_split(df, test_size = 0.2)
print("Training size: {}, Test size: {}".format(len(train), len(test)))

Training size: 12209169, Test size: 3052293


In [12]:
# train.shape
test.shape
# train.head()

(3052293, 11)

In [13]:
# feature_cols = ['code_module','code_presentation', 'assessment_type','weight','date_submitted','date','score','is_banked', 'num_of_prev_attempts']
# feature_cols = ['gender','region','highest_education','age_band','num_of_prev_attempts',
#                 'studied_credits','disability','activity_type','sum']
feature_cols =['code_module', 'code_presentation', 'assessment_type','score',
               'imd_band', 'id_site', 'activity_type', 'sum']

In [14]:
X_train = train[feature_cols]
y_train = train['final_result']

X_test = test[feature_cols]
y_test = test['final_result']

# Decision Tree

In [11]:
#feature importance
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier


In [12]:
model = ExtraTreesClassifier()
model.fit(X_train, y_train)
# display the relative importance of each attribute
print(model.feature_importances_)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


KeyboardInterrupt: 

In [30]:


feature_cols =['score', 'id_site', 'sum', 'imd_band','weight' ]
X_train = train[feature_cols]
y_train = train['final_result']

X_test = test[feature_cols]
y_test = test['final_result']

In [33]:
# train.shape
#test.shape
# train.head
X_train.head()

,score,id_site,sum,imd_band,weight
4038054,76.0,779707,1,6.0,0.0
11637883,90.0,768628,2,5.0,18.0
8428997,74.0,909229,34,7.0,0.0
11258314,52.0,716683,1,4.0,8.0
13496268,100.0,897121,1,9.0,0.0


In [18]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=17)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=17, splitter='best')

In [19]:
tree_predictions = tree.predict(X_test)
accuracy_score(y_test, tree_predictions)

0.6487237627580315

In [20]:
print('Accuracy is ' + str(round(accuracy_score(y_test, tree_predictions),3)))

Accuracy is 0.649


In [23]:
from sklearn.metrics import classification_report

y_pred=tree_predictions
y_true=y_test

from sklearn.metrics import f1_score
metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
report = classification_report(y_true,y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00    168859
           1       0.86      0.03      0.05    379675
           2       0.66      0.95      0.78   1946515
           3       0.54      0.21      0.30    557244

    accuracy                           0.65   3052293
   macro avg       0.52      0.30      0.28   3052293
weighted avg       0.62      0.65      0.56   3052293



In [26]:
tree_params = {'max_depth': range(2,10)}
locally_best_Tree = GridSearchCV(tree, tree_params, cv=4, n_jobs=-1, verbose=True)
locally_best_Tree.fit(X_train, y_train)

Fitting 2 folds for each of 8 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:  3.8min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy', max_depth=4,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1, param_grid={'max_depth': range(2, 10)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=Non

In [27]:
# locally_best_tree.best_params_
tuned_tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=17)
tuned_tree.fit(X_train, y_train)
tuned_tree_predictions = tuned_tree.predict(X_test)
accuracy_score(y_test, tuned_tree_predictions)

0.6487658135159982

In [28]:
#confusion matrix
from sklearn.metrics import classification_report

y_pred=tuned_tree.predict(X_test)
y_true=y_test


from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
report = classification_report(y_true,y_pred)

print(report)

//anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     84700
           1       0.86      0.02      0.05    189602
           2       0.66      0.95      0.78    973473
           3       0.54      0.21      0.30    278372

    accuracy                           0.65   1526147
   macro avg       0.51      0.30      0.28   1526147
weighted avg       0.62      0.65      0.56   1526147

